In [46]:
import requests
import util.item_analysis as ui
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

# 예
market = ['심리 상담 센터']
addr = '서울시 영등포구'

# 고객이 입력한 상품을 파는 매장을 리스트로 가져오기
market_list = market

In [47]:
# 고객이 입력한 주소 ['시', '구'] 또는 ['군', '면', '리'] 가져오기
addr_list = addr.split(' ')
addr_list1 = []
addr_list2 = []
for i in addr_list:
    if i[-1] == '시':
        addr_list1.append(i)
    elif i[-1] == '구':
        addr_list1.append(i)
    elif i[-1] == '군':
        addr_list2.append(i)
    elif i[-1] == '면':
        addr_list2.append(i)
    elif i[-1] == '리':
        addr_list2.append(i)

addr1 = ' '.join(addr_list1)
addr2 = ' '.join(addr_list2)

if addr1 != '':
    addr = addr1
else:
    addr = addr2
addr

'서울시 영등포구'

In [48]:
# 고객이 입력한 주소 좌표 구하기
try:
    with open('keys/카카오api.txt') as file:
        kakao_key = file.read()

    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    url = f'{base_url}?query={quote(addr)}'
    result = requests.get(url, headers=header).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
except:
    print('주소 형식이 올바르지 않습니다.')

In [49]:
lat, lng

(37.5263635705036, 126.896273923703)

In [50]:
# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)

In [51]:
# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

In [52]:
# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)

In [53]:
# '상세 주소 화살표' 클릭
try:
    juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
    juso_way[0].click()
    time.sleep(1)
except:
    print('근처에 매장이 없습니다')

In [54]:
# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source)
market_list = soup.find('ul', class_='eDFz9')
mart = market_list.find('li', recursive=False)
mart

<li class="VLTHu" data-laim-exp-id="undefined"><div class="qbGlu"><div class="ouxiq"><a class="P7gyV" href="https://m.place.naver.com/place/1069666803?entry=ple" role="button"><div class="ApCpt"><div class="place_bluelink C6RjW"><span class="YwYLL">닥터앤맘</span><span class="YzBgS">아동,청소년상담</span></div></div></a><a class="P7gyV" href="https://m.place.naver.com/place/1069666803?entry=ple" role="button"><div class="iHXbN"><div class="Gvf9B"><span class="XP3ml yTY83">영업 중</span><span class="XP3ml">18:00에 영업 종료</span></div></div></a><div class="d7iiF"><div class="KgfA6"><span class="lWwyx NVngW"><span class="place_blind">현재 위치에서</span>220m</span><span class="lWwyx"><a aria-expanded="true" aria-haspopup="true" class="uFxr1" href="#" role="button"><span class="Pb4bU">서울 영등포구 당산동3가</span><span class="_44_8"><span class="place_blind">상세주소 닫기</span><svg aria-hidden="true" class="Y2sSu" viewbox="0 0 13 8" xmlns="http://www.w3.org/2000/svg"><path d="M6.5 7.7L.1 1.4.8.7l5.6 5.6L12 .7l.7.7-6.2 6.3z"><

In [55]:
('//*[@id="_list_scroll_container"]/div/div/div[2]/ul/li[1]/div[1]/div/div/div/div/div')

'//*[@id="_list_scroll_container"]/div/div/div[2]/ul/li[1]/div[1]/div/div/div/div/div'

In [57]:
market_list.select_one('div.AbTyi').get_text().split('복사')[1][2:]

'서울특별시 영등포구 당산동3가 558-5'

In [58]:
results = []

# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)


# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)

# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source, 'html.parser')
center_list = soup.find('ul', class_='eDFz9')
for i in range(len(center_list.select('li', recursive=False))):
   
    if center_list:
        center = center_list.select('li', recursive=False)[i]

        if center:
            # 현재 위치에서 마켓 거리
            c_distance_element = center.select_one('span.lWwyx.NVngW')
            c_distance = c_distance_element.get_text().split('서')[1] if c_distance_element else 'N/A'

            # 마켓 이름
            if center.select_one('span.YwYLL'):
                c_title = center.select_one('span.YwYLL').get_text() 
            else:
                c_title = 'N/A'   

             # '상세 주소 화살표' 클릭
            try:
                juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
                juso_way[i].click()
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                center_list = soup.find('ul', class_='eDFz9')
                # 마켓 도로명 주소
                if  center_list.select_one('div.zZfO1'):
                    if center_list.select_one('div.zZfO1').get_text()[:2] == '도로':
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[0][3:]
                    elif center_list.select_one('div.zZfO1').get_text()[:2] == '지번':
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[1][2:]
                    else:
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[1][2:]
                else:
                    c_addr = 'N/A' 
                time.sleep(1)
            except:
                print('근처에 매장이 없습니다')
# 딕셔너리 형태로 저장
            center_data = {
                '거리': c_distance,
                '매장명': c_title,
                '주소': c_addr
            }
            # 딕셔너리 형태로 저장한 것을 리스트에 저장 (3가지 마켓을 넣어야 하므로)
            results.append(center_data)

        else:
            print('마켓 정보를 찾을 수 없습니다.')
    else:
        print('마켓 리스트를 찾을 수 없습니다.')


results

[{'거리': '220m', '매장명': '닥터앤맘', '주소': 'N/A'},
 {'거리': '300m', '매장명': '같이심리상담센터', '주소': 'N/A'},
 {'거리': '350m', '매장명': '당신의 마음거울', '주소': 'N/A'},
 {'거리': '380m', '매장명': '다빈치심리상담센터', '주소': 'N/A'},
 {'거리': '530m', '매장명': '서울예술심리상담센터', '주소': 'N/A'},
 {'거리': '590m', '매장명': '아하서울시립청소년성문화센터', '주소': 'N/A'},
 {'거리': '740m', '매장명': '힐마인드언어심리상담센터', '주소': 'N/A'},
 {'거리': '750m', '매장명': '세계로최면심리상담센터', '주소': 'N/A'},
 {'거리': '790m', '매장명': '남부Wee센터', '주소': 'N/A'},
 {'거리': '890m', '매장명': '풀빛옹이 심리상담센터', '주소': 'N/A'},
 {'거리': '950m', '매장명': '성모심리상담센터', '주소': 'N/A'},
 {'거리': '990m', '매장명': '최정금심리학습클리닉', '주소': 'N/A'},
 {'거리': '1.0km', '매장명': '연세오푸스심리상담센터', '주소': 'N/A'},
 {'거리': '1.0km', '매장명': '코리안민트심리상담센터', '주소': 'N/A'},
 {'거리': '1.0km', '매장명': '다움심리상담센터', '주소': 'N/A'},
 {'거리': '1.0km', '매장명': '분석심리학연구소', '주소': 'N/A'},
 {'거리': '1.1km', '매장명': '인애', '주소': 'N/A'},
 {'거리': '1.1km', '매장명': '보보애 심리상담센터', '주소': 'N/A'},
 {'거리': '1.1km', '매장명': '수아람심리상담센터', '주소': 'N/A'},
 {'거리': '1.1km', '매장명': '마음 in 심리상담센터', '주소

In [ ]:
results = []
# 현재 위치에서 마켓 거리
m_distance = mart.select_one('span.lWwyx.NVngW').get_text().split('서')[1]
# 마켓 이름
m_title = mart.select_one('span.place_bluelink.YwYLL').get_text()
# 마켓 도로명 주소
if mart.select_one('div.zZfO1').get_text()[:2] == '지번':
    m_addr = mart.select_one('div.zZfO1').get_text()[2:-2]
else:
    m_addr = mart.select_one('div.zZfO1').get_text()[3:-2]
# 딕셔너리 형태로 저장
market_data = {
    '거리': m_distance,
    '매장명': m_title,
    '주소': m_addr
}
# 딕셔너리 형태로 저장한 것을 리스트에 저장 (3가지 마켓을 넣어야 하므로)
results.append(market_data)
results

AttributeError: 'NoneType' object has no attribute 'get_text'